# Руководство по использованию модели artm.LDA в библиотеке BigARTM.

Автор - **Мурат Апишев** (great-mel@yandex.ru)

Модель LDA модуля artm предназначена для использования пользователями с минимальными представлениями о тематическом моделировании и теории аддитивной регуляризации тематических моделей. Она представляет собой урезанную версию модели artm.ARTM с предопрделёнными регуляризаторами и метриками качества. Возможностей artm.LDA достаточно для обучения соответствующей модели с регуляризаторами сглаживания/разреживания матриц $\Phi$ и $\Theta$ с помощью оффлайнового или онлайнового алгоритмов и извлечения результатов в виде самих матриц, а также метрик перплексии, разреженностей матриц и наиболее вероятных слов в каждой теме.

Информацию об интерфейсе artm.LDA можно найти здесь: http://bigartm.readthedocs.org/en/master/python_interface/lda.html

Проведём модельный эксперимент с использованием коллекции kos в формате UCI. О том, как пользоваться BatchVectorizer, что такое словари и т.п. можно прочесть по этой ссылке во вводном разделе: http://nbviewer.ipython.org/github/bigartm/bigartm-book/blob/master/ARTM_tutorial_RU.ipynb

Скачать коллекцию kos можно тут: https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/

Там же описано, что из себя представляет формат UCI Bag-Of-Words. Вам потребуются два файла: docword.kos.txt и vocab.kos.txt. Эти два файла надо положить в одну директорию с этим ноутбуком.

Импортируем модуль artm, создадим BatchVectorizer и запустим внутри него сборку словаря (если Вас интересует, что это такое, то об этом можно прочесть по ссылке выше, для понимания описываемого кода достаточно знать, что словарь содержит список всех уникальных слов коллекции с некоторой информацией, и нужен для инициализации модели и корректного подсчёта перплексии):

### 1 . Самая базовая часть.

In [1]:
import artm

print artm.version()

0.8.0


In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='.', data_format='bow_uci',
                                        collection_name='kos', target_folder='kos_batches')

Теперь создадим саму тематическую модель, указав число тем и параметр числа прохода по документу, гиперпараметры сглаживания матриц $\Phi$ и $\Theta$, а также используемый словарь. Кроме того, попросим сохранять матрицу $\Theta$, чтобы потом на неё можно было взглянуть.


Здесь же можно указать параметр num_processors, отвечающий за число потоков, которые будут использованы для вычислений на машине.

In [ ]:
lda = artm.LDA(num_topics=15, alpha=0.01, beta=0.001,
               num_document_passes=5, dictionary=batch_vectorizer.dictionary,
               cache_theta=True)

Запустим процесс обучения с помощью оффлайнового алгоритма:

In [ ]:
lda.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

Всё, на этом всё обучение окончено. Дальше можно повторять последнюю команду, редактировать параметры модели и т.п.

Уже сейчас можно посмотреть на результаты моделирования. Например, на финальные значения разреженностей матриц:

In [ ]:
lda.sparsity_phi_last_value
lda.sparsity_theta_last_value

Или на значения перпелексии на каждой итерации прохода по коллекции:

In [ ]:
lda.perplexity_value

Наконец, можно посмотреть на наиболее вероятные слова в каждой теме. Они выдаются в виде списка списков строк (каждый внутренний список соответствуюет одной теме, по порядку). Выведем их, предварительно отформатировав:

In [ ]:
top_tokens = lda.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
    print 'Topic #{0}: {1}'.format(i, token_list)

Можно получить доступ к матрицам с помощью следующих вызовов:

In [ ]:
phi = lda.phi_
theta = lda.get_theta()

### 2. Чуть больше деталей.

Опишем ещё ряд возможностей модели artm.LDA.

Во-первых - это построение матрицы $\Theta$ для новых документов при наличии обученной модели:

In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='kos_batches_test')
theta_test = lda.transform(batch_vectorizer=test_batch_vectorizer)

Во-вторых, в том случае, если требуется производить различную регуляризацию каждой из тем в матрице $\Phi$, вместо скалярного значения beta можно задать список гиперпараметров, длиной в число тем, и каждая тема будет регуляризована со своим гиперпараметром:

In [ ]:
beta = [0.1] * num_topics  # change as you need
lda = artm.LDA(num_topics=15, alpha=0.01, beta=beta, num_document_passes=5, dictionary=dictionary, cache_theta=True)

### Заключение

Эта модель - самый простой способ использования BigARTM. От Вас не требуется ничего, кроме преобразования данных в формат BigARTM с помощью BatchVectorizer, у которого есть подробное описание в руководстве и документации, и запуска нескольких описанных выше строк кода.

В случае необходимости более продвинутого использования библиотеки, нужно воспользоваться модель artm.ARTM, подробное описание которой есть в основном руководстве.